In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


<link rel="stylesheet" href="../../../common/dhds.css">
<div class="Table">
    <div class="Row">
        <div class="Cell grey left"> <img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M6/CLASE_48_Series_de_Tiempo_1/Presentacion/img/M6_CLASE_48_portada.gif" align="center" width="70%"/></div>
        <div class="Cell right">
            <div class="div-logo"><img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/common/logo_DH.png" align="center" width=70% /></div>
            <div class="div-curso">DATA SCIENCE</div>
            <div class="div-modulo">MÓDULO 6</div>
            <div class="div-contenido">Series de Tiempo 1

</div>
        </div>
    </div>
</div>

### Agenda

---

- Series de tiempo.
   - Definición.
   - Componentes.
   - Descomposición.
---

- Modelos básicos para pronósticos.
   - Random Walk.
   - Media constante.
   - Tendencia lineal.
   - Tendencia cuadrática.
   - Tendencia con transformación logarítmica con y sin Estacionalidad mensual.
   - Media movil.
   - Single Exponential Smoothing.

<div class="div-dhds-fondo-1"> Introducción
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M3/CLASE_21_Regresion_Lineal_Simple/Presentacion/img/M3_CLASE_21_separador.png" align="center" />

</div>

### Introducción

---
Una **serie de tiempo** es un conjunto de observaciones tomadas en *intervalos regulares* (lags), **ordenadas** por el momento en que se produjeron.

**Time series analysis** (*análisis de series de tiempo*) comprende métodos para *proyectar la evolución*, obtener *estadísticas y otras características* de una variable a lo largo del tiempo.

**Time series forecasting** (*Pronóstico de series de tiempo*) es el uso de los modelos para predecir futuros valores.

La serie de tiempo se proyecta como una *variable* (eje y) en *función del tiempo* (eje x). 

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M6/CLASE_48_Series_de_Tiempo_1/Presentacion/img/M6_CLASE_48_001_serie.JPG" alt="serie" width=50% height=35% align="center"/>

Existen muchos ejemplos de datos que se pueden expresar como series de tiempo:

- Variables macroeconómicas (PBI, inflación, reservas del BCRA...).

- Movimientos comerciales (ventas, compras, pagos a proveedores...).

- Datos de producción (horas de horno, material consumido...).

- Consumo energético.

- Activos financieros.

- Variables sociales (mortalidad infantil, pobreza...).

### Componentes de una serie de tiempo

---

Podemos encontrar en una serie de tiempo **cuatro componentes**:

- **Tendencia** (Trend): es el componente *permanente*, el efecto persistente en el tiempo.

- **Estacionalidad** (Seasonality): es un patrón estacional que se *repite con regularidad*. 

- **Componente aleatoria** (Residual - Remainder): son shocks que no presentan un efecto duradero. Se los define también como ruido o movimientos random.

- **Ciclos** (Cycle): otro tipo de dinámica no capturada por la tendencia o estacionalidad. 

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M6/CLASE_48_Series_de_Tiempo_1/Presentacion/img/M6_CLASE_48_002_componentes.jpg" alt="componentes" width=50% height=30% align="center"/>

El módulo `statsmodels.api`, método `tsa.seasonal_decompose` genera gráficos con: *la serie de tiempo, la tendencia, la estacionalidad y el componente aleatorio*.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
#import matplotlib as mpl
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt

from scipy import stats
from statistics import mode

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_excel("../Data/Supertienda.xls")
mobiliario = df.loc[df['Categoria'] == 'Mobiliario']

In [ ]:
mobiliariogb = mobiliario.groupby('Fecha_pedido')['Ventas'].sum().reset_index()
mobiliariogb = mobiliariogb.set_index('Fecha_pedido')
y = mobiliariogb['Ventas'].resample('MS').mean()

In [ ]:
import statsmodels.api as sm
plt.figure(figsize=(16,10))
decomposition = sm.tsa.seasonal_decompose(y, model='additive')
decomposition.plot()
plt.show()

### Descomposición de una serie de tiempo

---

Podemos pensar en descomponer una serie de tiempo en sus componentes. Básicamente, existen dos formas:

- **Descomposición aditiva**:$$ y_t = T_t + S_t + C_t + \varepsilon_t. $$

donde T es la tendencia, S es la estacionalidad, C es el ciclo y $ \varepsilon $ es el error aleatorio.

- **Descomposición multiplicativa**: $$ y_t = T_t * S_t * C_t * \varepsilon_t. $$

- La descomposición aditiva es útil cuando la variación estacional se *mantiene relativamente constante* (lineal).

  O dicho de otra forma, la altura de sus picos es constante. 

- La descomposición multiplicativa es útil cuando la *tendencia crece* y la *amplitud de la variación estacional aumenta* (exponencial).

  O por el contrario, cuando la tendencia y la amplitud decrecen.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M6/CLASE_48_Series_de_Tiempo_1/Presentacion/img/M6_CLASE_48_003_descomposicion.png" alt="descomposicion" width=80% height=50% align="center"/>


<div class="div-dhds-fondo-1"> Modelos básicos para pronósticos
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M3/CLASE_21_Regresion_Lineal_Simple/Presentacion/img/M3_CLASE_21_separador.png" align="center" />

</div>

### Introducción

---
Estudiamos modelos aplicados a los componentes de las series para generar predicciones.

Para cada modelo:

- Lo *definimos*. 

- Lo *ajustamos* a los datos de train.

- Lo *evaluamos* usado RMSE sobre los datos de test.

- Lo *comparamos* con otros modelos.

Recordemos que el **error cuadrático medio raíz (RMSE)** compara los valores predichos con los valores observados.

$${RMSE} = \sqrt{{\frac {1}{n}} \sum _{i=1}^{n}(y_{i}-\hat{y_{i}})^2}$$

In [ ]:
def RMSE(predicted, actual):
    mse = (predicted - actual) ** 2
    rmse = np.sqrt(mse.sum() / mse.count())
    return rmse

<div class="div-dhds-fondo-1">  Dataset
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M3/CLASE_21_Regresion_Lineal_Simple/Presentacion/img/M3_CLASE_21_separador.png" align="center" />

</div>

### Dataset

---

Vamos a usar datos de ventas de una empresa multinacional en varios paises, entre los años 2018 y 2021.

Consideramos las ventas de la categoría Muebles.

In [ ]:
df = pd.read_excel("../Data/Supertienda.xls")
mobiliario = df.loc[df['Categoria'] == 'Mobiliario']
mobiliario.head(2)

Mensualizamos las ventas por fecha de pedido.

Primero agrupamos las ventas por el atributo 'Fecha_pedido'. Y *creamos un índice sobre el atributo* para indicar la fecha que se usa en la serie de tiempo.

In [ ]:
mobiliariogb = mobiliario.groupby('Fecha_pedido')['Ventas'].sum().reset_index()
mobiliariogb = mobiliariogb.set_index('Fecha_pedido')

La función `resample` de Pandas, permite hacer *downsampling*, llevar la serie a un espacio de tiempo más amplio. 

En nuestro caso, pasamos de días a meses; `MS` indica "a comienzo de mes".

Y con la función `mean()` agrupamos todas las ventas del mes y le calculamos el promedio.

In [ ]:
df_ventas = mobiliariogb['Ventas'].resample('MS').mean().to_frame(); df_ventas.head(2)

In [ ]:
plt.figure(figsize=(4,3)); df_ventas.plot(); plt.show()

### Dataset

---
Dividimos el dataset en train y test. Al ser una serie de tiempo, tenemos que poner *`shuffle=False`* (que no mezcle las observaciones), creando una continuidad entre los sets de entrenamiento y de testeo.

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_ventas, test_size=12, random_state=42, shuffle=False)

Para train considera los primeros tres años, y para test el último, 2021.

In [ ]:
# Vemos los últimos datos del set de entrenamiento:
df_train.tail(2)

In [ ]:
# Vemos los primeros datos del set de testeo:
df_test.head(2)

<div class="div-dhds-fondo-1">  Random Walk
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M3/CLASE_21_Regresion_Lineal_Simple/Presentacion/img/M3_CLASE_21_separador.png" align="center" />

</div>

### Definición

---

Decimos que un proceso $ {Y_t} $ es **random walk** (sigue una trayectoria al azar) si:

$$ \ {Y_t} = Y_ {t-1} + \varepsilon,  $$
   donde $\varepsilon$ es **ruido blanco** (White Noise). 

El *ruido blanco* es la parte de la serie que no se puede predecir a partir de la evidencia.

Si el modelo de *random walk* sigue a la tendencia, la representamos con una constante $d$, llamada **drift** (deriva). 

Se genera un modelo **random walk with drift** (con deriva):

$$ \ {Y_t} = Y_ {t-1} + d + \varepsilon $$


- La serie de White Noise $ \varepsilon_t $ cumple con:

   - Media (μ) = 0 y la varianza constante. Se puede escribir como $ WN(0,σ²)$ .
   
   - Es completamente random. Por eso se lo llama ruido.
   
   - Son observaciones independientes identicamente distribuidas (i.i.d.).
   
   - Tienen cero correlación entre ellas.
   
Si las observaciones siguen una distribución normal, se lo llama Gaussian White Noise Gaussiano. 

### Predicción

---

El caso más simple es suponer que el valor de la serie Yₜ se predice como el valor previo Yₜ₋₁, es decir, que tendremos valor de cambio cero.

Se llama *pronóstico ingenuo* (naive).

Un ejemplo de la vida real puede ser la estimación de los valores de las acciones.

Vamos a realizar este modelo.

Generamos el desplazamiento de nuestro target en el set de entrenamiento (lag=1):

In [ ]:
df_train["VentasShift1"] = df_train.Ventas.shift()

# La primera observación nos va a quedar en nan, la reemplazamos por el valor siguente:
df_train["VentasShift1"].fillna(method='bfill', inplace=True)
df_train.head(3)

Generamos el desplazamiento en el set de test (lag=1):

In [ ]:
df_test["VentasShift1"] = df_test.Ventas.shift()

df_test["VentasShift1"].fillna(method='bfill', inplace=True)
df_test.head(3)

Definimos como *random walk* al desplazamiento en un valor (shift=1).

In [ ]:
df_train["RandomWalk"] = df_train.VentasShift1
df_train.plot(kind="line", y = ["Ventas", "RandomWalk"]);

La predicción sobre test es simplemente la última observación registrada en train. 

In [ ]:
df_test["RandomWalk"] = pd.Series(df_train["Ventas"][-1], index=df_test.index)

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.plot(kind="line", y = ["Ventas", "RandomWalk"]);

In [ ]:
# Calculamos el RMSE y almacenamos los resultados
df_Results = pd.DataFrame(columns = ["Model", "RMSE"])
df_Results.loc[0, "Model"] = "Random Walk"
df_Results.loc[0, "RMSE"] = RMSE(df_test.RandomWalk, df_test.Ventas)
df_Results

<div class="div-dhds-fondo-1">  Media Constante
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M3/CLASE_21_Regresion_Lineal_Simple/Presentacion/img/M3_CLASE_21_separador.png" align="center" />

</div>

### Definición

---

La **media constante** es el modelo más básico e ingenuo de todos. 

Consiste simplemente en tomar *la media del dataset de train* y usarlo para predecir. 

In [ ]:
model_mean_pred = df_train['Ventas'].mean()
model_mean_pred

La media se aplica tanto a train como a test. 

In [ ]:
df_train_mc = df_train.copy()
df_train_mc['Mean'] = model_mean_pred

df_test_mc = df_test.copy()
df_test_mc['Mean'] = model_mean_pred

Visualmente ya se observa que el método no predice correctamente. 

In [ ]:
fig, axes = plt.subplots(1, 2,figsize=(12,6))
df_train_mc.plot(ax=axes[0], y = ["Ventas","Mean"],title='Train')
df_test_mc.plot(ax=axes[1],y = ["Ventas","Mean"], title='Test')
plt.show()

Calculamos la performance con RMSE y la guardamos para comparar con otros modelos.

In [ ]:
model_RMSE = RMSE(df_test_mc.Mean, df_test_mc.Ventas)

In [ ]:
df_Results.loc[1, "Model"] = "Mean"
df_Results.loc[1, "RMSE"] = model_RMSE
df_Results.head()

<div class="div-dhds-fondo-1">  Tendencia Lineal
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M3/CLASE_21_Regresion_Lineal_Simple/Presentacion/img/M3_CLASE_21_separador.png" align="center" />

</div>

### Definición

---

Recordemos que el componente de la serie **tendencia** corresponde a la **evolución de largo plazo**. 

Podemos modelar la tendencia de diferentes maneras, de acuerdo a su dinámica. 

- La **tendencia lineal** representa una recta para predecir.

  $$ T_t = \beta_0 + \beta_1 TIME_t + \varepsilon_t. $$

Donde *TIME*, llamada “dummy de tiempo”, es una secuencia que representa al tiempo.

In [ ]:
df_ventas["timeIndex"] = pd.Series(np.arange(len(df_ventas['Ventas'])), index=df_ventas.index)
df_ventas.timeIndex.head(3)

Volvemos a hacer los datasets de train y test con la nueva variable.

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_ventas, test_size=12, random_state=42, shuffle=False)

### Predicción

---
Para generar la recta, usamos un modelo de *regresión lineal* entre "Ventas" y "timeIndex". 

El módulo `statmodels` nos permite ajustar modelos con su API, usando fórmulas estilo R.

La sentencia:
   `smf.ols(formula='Ventas ~ timeIndex', data = df_train).fit()`

Nos indica que haremos regresión lineal (`ols`), con dos variables (`formula`).
    
Para mas detalles,  [ver aquí](http://www.statsmodels.org/dev/examples/notebooks/generated/formulas.html).

In [ ]:
import statsmodels.api as sm
sm.OLS.from_formula

In [ ]:
import statsmodels.formula.api as smf
model_linear = smf.ols(formula='Ventas ~ timeIndex', data = df_train).fit()

Generamos las predicciones del modelo sobre *train y test*.

In [ ]:
df_train["LinearTrend"] = model_linear.predict(df_train.timeIndex)
df_test["LinearTrend"] = model_linear.predict(df_test.timeIndex)

Visualmente observamos que la recta ajusta mejor que la media constante. 

In [ ]:
fig, axes = plt.subplots(1, 2,figsize=(12,6))
df_train.plot(kind = "line", y = ["Ventas","LinearTrend"],ax=axes[0],title='Train');
df_test.plot(kind = "line", y = ["Ventas","LinearTrend"],ax=axes[1],title='Test');
plt.show()

Calculamos la performance con RMSE. Y confirmamos analíticamente que es mejor que Mean.

In [ ]:
model_RMSE = RMSE(df_test.LinearTrend, df_test.Ventas)

In [ ]:
df_Results.loc[2, "Model"] = "LinearTrend"
df_Results.loc[2, "RMSE"] = model_RMSE
df_Results.head()

`statmodels` nos permite ver los resultados de la regresión lineal.

In [ ]:
model_linear.summary()

<div class="div-dhds-fondo-1"> Tendencia cuadrática
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_34_CART/Presentacion/img/M5_CLASE_34_separador.png" align="center" />

</div>

### Definición

---
Si la tendencia presenta una dinámica no lineal, podemos incluir un **término cuadrático** al modelo. 

Lo obtenemos simplemente elevando al cuadrado a la dummy de tiempo:

$$ T_t = \beta_0 + \beta_1 TIME_t + \beta_2 TIME^{2}_t +\varepsilon_t. $$

In [ ]:
df_ventas["timeIndex_sq"] = df_ventas["timeIndex"]**2
df_ventas.timeIndex_sq.head(3)

Volvemos a hacer los datasets de train y test con la nueva variable.

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_ventas, test_size=12, random_state=42, shuffle=False)

### Predicción

---
En este modelo, incorporamos el valor cuadrático a la fórmula.

In [ ]:
import statsmodels.formula.api as smf
model_quadratic = smf.ols('Ventas ~ timeIndex + timeIndex_sq', data = df_train).fit()

Generamos las predicciones del modelo sobre *train y test*.

In [ ]:
df_train["QuadraticTrend"] = model_quadratic.predict(df_train[["timeIndex","timeIndex_sq"]])
df_test["QuadraticTrend"] = model_quadratic.predict(df_test[["timeIndex","timeIndex_sq"]])

Observamos visualmente las predicciones.

In [ ]:
fig, axes = plt.subplots(1, 2,figsize=(12,6))
df_train.plot(kind = "line", y = ["Ventas","QuadraticTrend"],ax=axes[0],title='Train');
df_test.plot(kind = "line", y = ["Ventas","QuadraticTrend"],ax=axes[1],title='Test');
plt.show()

Calculamos la performance con RMSE. En este dataset, sigue ajustando mejor la tendencia lineal.

In [ ]:
model_RMSE = RMSE(df_test.QuadraticTrend, df_test.Ventas)

In [ ]:
df_Results.loc[3, "Model"] = "QuadraticTrend"
df_Results.loc[3, "RMSE"] = model_RMSE
df_Results.head()

In [ ]:
model_quadratic.summary()

<div class="div-dhds-fondo-1"> Tendencia con transformación logarítmica
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_34_CART/Presentacion/img/M5_CLASE_34_separador.png" align="center" />

</div>

### Definición

---
En algunos casos, la varianza de la serie aumenta con el paso del tiempo. 

Hacer una **transformación logarítmica** de la serie puede ayudar a estabilizar la varianza.

- Generamos el modelo *sobre el logaritmo de las ventas*, y sus predicciones.

- *Volvemos a transformar* las predicciones a un valor de ventas, con una función exponencial.

- Este último valor será la *predicción final*, el cual evaluamos.

Trasformamos en logaritmo las ventas y las graficamos.

In [ ]:
df_train['log_Ventas'] = np.log(df_train['Ventas'])
df_test['log_Ventas'] = np.log(df_test['Ventas'])

In [ ]:
fig, axes = plt.subplots(1, 2,figsize=(10,5))
df_train.plot(kind = "line", y = ["log_Ventas"],ax=axes[0],title='Train');
df_test.plot(kind = "line", y = ["log_Ventas"],ax=axes[1],title='Test');
plt.show()

### Predicción

---
Generamos el modelo de regresión lineal usando el logaritmo de las ventas.

In [ ]:
import statsmodels.formula.api as smf
model_log = smf.ols('log_Ventas ~ timeIndex ', data = df_train).fit()

Y las predicciones del modelo sobre *train y test*.

In [ ]:
df_train["LogTrend"] = model_log.predict(df_train[["timeIndex"]])
df_test["LogTrend"] = model_log.predict(df_test[["timeIndex"]])

Recuperamos el valor predicho de la venta aplicando la función exponencial sobre las predicciones.

In [ ]:
df_train['back_LogTrend'] = np.exp(df_train['LogTrend'])
df_test['back_LogTrend'] = np.exp(df_test['LogTrend'])

Observamos visualmente las ventas predichas vs el valor real.

In [ ]:
fig, axes = plt.subplots(1, 2,figsize=(10,5))
df_train.plot(kind = "line", y = ["Ventas","back_LogTrend"],ax=axes[0],title='Train');
df_test.plot(kind = "line", y = ["Ventas","back_LogTrend"],ax=axes[1],title='Test');
plt.show()

Calculamos la performance con RMSE. El modelo ajusta mejor que los anteriores para este dataset.

In [ ]:
model_RMSE = RMSE(df_test.back_LogTrend, df_test.Ventas)

In [ ]:
df_Results.loc[4, "Model"] = "back_LogTrend"
df_Results.loc[4, "RMSE"] = model_RMSE
df_Results.head()

In [ ]:
model_log.summary()

<div class="div-dhds-fondo-1"> Transformación logarítmica <br/>  + estacionalidad mensual
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_34_CART/Presentacion/img/M5_CLASE_34_separador.png" align="center" />

</div>

### Definición

---

Recordemos que la  **estacionalidad** (Seasonality) es un patrón estacional que se *repite con regularidad*. 

Podemos modelar la estacionalidad utilizando variables dummy. En el siguiente ejemplo modelamos una estacionalidad trimestral:

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M6/CLASE_48_Series_de_Tiempo_1/Presentacion/img/M6_CLASE_48_004_estacionalidad.png" alt="estacionalidad" width=50% height=30% align="center"/>

$$ S_t = \sum_{i=1}^{s} \gamma_iD_it  + \varepsilon_t. $$

Se pueden crear otros tipos de dummies, por ejemplo por días de vacaciones/feriados, etc.

Establecemos variables dummy para los meses de la fecha de pedido de las ventas.

In [ ]:
# Creamos columna con el mes
df_train['month'] = [d.strftime('%b') for d in df_train.index]
df_test['month']  = [d.strftime('%b') for d in df_test.index]

In [ ]:
# Creamos las variables dummies de mes
dummies_mes_train = pd.get_dummies(df_train['month'])
dummies_mes_test = pd.get_dummies(df_test['month'])

In [ ]:
# Hacemos el join entre el DataFrame con la serie de tiempo y las dummies
df_train = df_train.join(dummies_mes_train)
df_test = df_test.join(dummies_mes_test)

In [ ]:
df_train.head(4)

### Predicción

---
Generamos el modelo de regresión lineal como en el caso anterior, agregando al logaritmo de las ventas, la estacionalidad (las variables dummies de mes).

In [ ]:
model_log_est = smf.ols('log_Ventas ~ timeIndex + Apr + Aug + Dec + Feb + Jan + Jul + Jun + Mar + May + Nov + Oct + Sep',\
                          data = df_train).fit()

Y las predicciones del modelo sobre *train y test*.

In [ ]:
df_train['model_log_est'] = model_log_est.predict(df_train[["timeIndex","Apr","Aug", "Dec", "Feb", "Jan",\
                                               "Jul", "Jun", "Mar", "May","Nov", "Oct", "Sep"]])

df_test['model_log_est'] = model_log_est.predict(df_test[["timeIndex","Apr","Aug", "Dec", "Feb", "Jan",\
                                               "Jul", "Jun", "Mar", "May","Nov", "Oct", "Sep"]])

Recuperamos el valor predicho de la venta aplicando la función exponencial sobre las predicciones.

In [ ]:
df_train['back_LogEstTrend'] = np.exp(df_train['model_log_est'])
df_test['back_LogEstTrend'] = np.exp(df_test['model_log_est'])

Observamos visualmente las ventas predichas vs el valor real.

In [ ]:
fig, axes = plt.subplots(1, 2,figsize=(10,5))
df_train.plot(kind = "line", y = ["Ventas","back_LogEstTrend"],ax=axes[0],title='Train');
df_test.plot(kind = "line", y = ["Ventas","back_LogEstTrend"],ax=axes[1],title='Test');
plt.show()

Calculamos la performance con RMSE. El modelo ajusta mucho mejor que todos los anteriores.

In [ ]:
model_RMSE = RMSE(df_test.back_LogEstTrend, df_test.Ventas)

In [ ]:
df_Results.loc[5, "Model"] = "back_LogEstTrend"
df_Results.loc[5, "RMSE"] = model_RMSE
df_Results.head()

In [ ]:
model_log_est.summary()

<div class="div-dhds-fondo-1"> Media Movil
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_34_CART/Presentacion/img/M5_CLASE_34_separador.png" align="center" />

</div>

### Definición

---
**Media movil** (Moving Average o Rolling Mean) es un *promedio* de los valores de una serie temporal *para un período de tiempo determinado* (la ventana de estimación).

El tamaño de la ventana de estimación $M$ es la cantidad de observaciones usadas para estimar.

Veamos el tipo de media movil llamado **media móvil simple** (Simple Moving Average).

que se expresa:
$$ {SMA_t} =\frac{1}{M}. (x_{t} + x_{t-1} + x_{t-2} + ... x_{M-(t-1)})$$
        donde el valor para el tiempo $t$, es un promedio de los M valores reales previos.

Es un *indicador de tendencia* ya que produce un **suavizado** sobre la serie temporal.

No la vamos a usar para predecir, pero puede ser útil como método de preprocesamiento de la serie.

Veamos la diferencia entre casos diarios y la media movil semanal de casos de covid-19.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M6/CLASE_48_Series_de_Tiempo_1/Presentacion/img/M6_CLASE_48_005_media_movil.JPG" alt="estacionalidad" width=80% height=50% align="center"/>

### Ejemplo

---
La forma más fácil para calcular **SMA** es usando el método de Pandas `Series.rolling`.

Las primeras filas contienen valores nulos, pues recién después de M observaciones se puede calcular el promedio. Pero usando el argumento `min_periods=1` podemos resolverlo.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M6/CLASE_48_Series_de_Tiempo_1/Presentacion/img/M6_CLASE_48_006_media_movil.png" alt="media_movil" width=70% height=25% align="center"/>

Observemos que a mayor ventana, más suave la serie calculada.

In [ ]:
df_ventas["sma_5"] = df_ventas.Ventas.rolling(5,min_periods=1).mean()
df_ventas["sma_15"] = df_ventas.Ventas.rolling(15,min_periods=1).mean()

In [ ]:
df_ventas.plot(kind = "line", y = ["Ventas","sma_5","sma_15"],title='SMA');

<div class="div-dhds-fondo-1"> Single Exponential Smoothing
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_34_CART/Presentacion/img/M5_CLASE_34_separador.png" align="center" />

</div>

### Definición

---
Recién vimos con *Media movil*, que los métodos de suavizado son *promedios ponderados* de observaciones pasadas. 

En el caso de **suavizamiento exponencial simple**, los pesos siguen una caída exponencial; le da más peso a las observaciones recientes y menos peso a las antiguas. 

Podemos expresarlo como:

$$ \widehat{y_t} =\alpha . y_{t-1}  + (1 - \alpha ) . \widehat{y}_{t -1} $$

Donde $\widehat{y_t}$ es el valor predicho para el tiempo $t$, que es un promedio ponderado entre el valor previo predicho $\widehat{y}_{t -1}$ y el valor actual $y_{t-1}$.

$\alpha$  se conoce como **smoothing parameter**; toma valores entre 0 y 1. 

Este método es adecuado para pronosticar datos *sin una tendencia ni un patrón estacional claro*.

Viendo la expresión:

$$ \widehat{y_t} =\alpha . y_{t-1}  + (1 - \alpha ) . \widehat{y}_{t -1} $$

- Si $\alpha$ es igual a 1, todas las predicciones *son iguales al último valor observado*. Se le llama un **método ingenuo (naive)**.

- Por otro lado, si consideramos la **media constante**, todas las predicciones *son iguales al promedio de los valores observados*.

En general se trabaja entre estos dos extremos. La expresión generalizada de **Single Exponential Smoothing**:

$$ \widehat{y_t} =\alpha . y_{t-1}  + \alpha . (1 - \alpha ) . y_{t-2}  + \alpha . (1 - \alpha )^2. y_{t-3} ...$$

Muestra que el peso de las observaciones **decrece en forma exponencial** a medida que son más antiguas.

Y $\alpha$ nos indica *el ratio de caída*. Más cerca de uno, decae más rápido.

### Predicción

---
Generamos tres modelos variando el  **smoothing factor $\alpha$** (0.3, 0.5 y 0.8).

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
model_exp_smoothing_3 = SimpleExpSmoothing(df_train.Ventas).fit(smoothing_level=0.3,optimized=False)
model_exp_smoothing_5 = SimpleExpSmoothing(df_train.Ventas).fit(smoothing_level=0.5,optimized=False)
model_exp_smoothing_8 = SimpleExpSmoothing(df_train.Ventas).fit(smoothing_level=0.8,optimized=False)

Vemos los resultados sobre los datos de train.

In [ ]:
df_train.plot(kind = "line", y = "Ventas",color='black')
model_exp_smoothing_3.fittedvalues.plot(label="alpha=0.3",color='b');
model_exp_smoothing_5.fittedvalues.plot(label="alpha=0.5",color='g');
model_exp_smoothing_8.fittedvalues.plot(label="alpha=0.8",color='r');
plt.legend();

Hacemos las predicciones del modelo sobre *test*.

In [ ]:
df_test["Simple_Smoothing_3"] = model_exp_smoothing_3.forecast(len(df_test))
df_test["Simple_Smoothing_5"] = model_exp_smoothing_5.forecast(len(df_test))
df_test["Simple_Smoothing_8"] = model_exp_smoothing_8.forecast(len(df_test))

El modelo con  $\alpha$ = 0.3 es el que mejor predice.

In [ ]:
model_RMSE_3 = RMSE(df_test["Simple_Smoothing_3"], df_test.Ventas)
model_RMSE_5 = RMSE(df_test["Simple_Smoothing_5"], df_test.Ventas)
model_RMSE_8 = RMSE(df_test["Simple_Smoothing_8"], df_test.Ventas)

In [ ]:
# Calculamos el RMSE y almacenamos los resultados
df_Results.loc[6, "Model"] = "Simple Smoothing a=0.3"; df_Results.loc[6, "RMSE"] = model_RMSE_3
df_Results.loc[7, "Model"] = "Simple Smoothing a=0.5"; df_Results.loc[7, "RMSE"] = model_RMSE_5
df_Results.loc[8, "Model"] = "Simple Smoothing a=0.8"; df_Results.loc[8, "RMSE"] = model_RMSE_8
df_Results

<div class="div-dhds-fondo-1"> Conclusiones
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_34_CART/Presentacion/img/M5_CLASE_34_separador.png" align="center" />

</div>

## Conclusiones

---

Una serie de tiempo tiene los componentes:

- **Tendencia**: el componente “permanente”, el efecto que persiste en el largo plazo.
- **Estacionalidad**: los movimientos periódicos de la serie.
- **Componente aleatoria**: son shocks que no presentan un efecto duradero.
- **Ciclos**: se entiende por cualquier tipo de dinámica no capturada por la tendencia o la estacionalidad. 

Para modelar la tendencia y la estacionalidad podemos usar *dummies de tiempo y estacionales*. 

Existen varios *modelos para predecir* los nuevos valores de la serie: Tendencia lineal, Tendencia Cuadrática, Transformación logarítmica con y sin estacionalidad mensual y Single Exponential Smoothing.

La *media movil* es útil para "suavizar" la serie.

<div class="div-dhds-fondo-1"> Hands-on
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M3/CLASE_21_Regresion_Lineal_Simple/Presentacion/img/M3_CLASE_21_separador.png" align="center" />

</div>

### Ejercicio

----

A partir del dataset de ventas, seleccionar la categoría de Tecnología y

- Generar la serie de tiempo sobre las ventas y graficarla.
- Graficar los componentes de la serie.
- Generar un modelo de Tendencia lineal y validarlo. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Ejercicio

----
A partir del dataset filtrado por Tecnología, generamos la serie de tiempo y la graficamos.

In [ ]:
df = pd.read_excel("../Data/Supertienda.xls")
#tecno = df.loc[df['Categoria'] == '.....']

In [ ]:
#tecno.head(2)

In [ ]:
#tecnogb = tecno.groupby('.....')['.....'].sum().reset_index()
#tecnogb = tecnogb.set_index('.....')

In [ ]:
#df_ventas_t = tecnogb['.....'].resample('..').mean().to_frame(); 
#df_ventas_t.head(2)

Graficamos la serie.

In [ ]:
#plt.figure(figsize=(4,3)); ........plot(); plt.show()

### Ejercicio

----
Graficamos los componentes de la serie de tiempo (Tendencia, Estacionalidad y la Componente aleatoria).

In [ ]:
import statsmodels.api as sm
plt.figure(figsize=(16,10))
#decomposition = sm.tsa.seasonal_decompose(....., .....)
#decomposition.plot()
plt.show()

### Ejercicio

----
Generamos el modelo de Tendencia Lineal: lo ajustamos sobre train, creamos las predicciones, las graficamos sobre train y test, y evaluamos el modelo con RMSE.

Previamente definimos la variable “dummy de tiempo”, la secuencia que representa al tiempo.

In [ ]:
#df_ventas_t["timeIndex"] = pd.Series(np.arange(len(......['.....'])), index=..........index)
#df_ventas_t.timeIndex.head(3)

Hacemos los datasets de train y test. Recordar que tenemos que usar `shuffle=False`.

In [ ]:
from sklearn.model_selection import train_test_split
#df_train_t, df_test_t = train_test_split(...., test_size=12, random_state=42,.......)

Creamos un modelo de *regresión lineal* entre "Ventas" y "timeIndex". 

In [ ]:
import statsmodels.formula.api as smf
#model_linear = smf.ols(formula='..... ~ ......', data = .....).fit()

Generamos las predicciones del modelo sobre *train y test*.

In [ ]:
#df_train_t["LinearTrend"] = model_linear.predict(.......timeIndex)
#df_test_t["LinearTrend"] = model_linear.predict(.......timeIndex)

Chequeamos visualmente las predicciones. 

In [ ]:
fig, axes = plt.subplots(1, 2,figsize=(12,6))
#df_train_t.plot(kind = "line", y = [".....","....."],ax=axes[0],title='Train');
#df_test_t.plot(kind = "line", y = [".....","......"],ax=axes[1],title='Test');
plt.show()

Calculamos la performance con RMSE.

In [ ]:
def RMSE(predicted, actual):
    mse = (predicted - actual) ** 2
    rmse = np.sqrt(mse.sum() / mse.count())
    return rmse

In [ ]:
#model_RMSE = RMSE(..... , .......)

In [ ]:
#df_Results = pd.DataFrame(columns = ["Model", "RMSE"])
#df_Results.loc[0, "Model"] = "LinearTrend"
#df_Results.loc[0, "RMSE"] = model_RMSE
#df_Results.head()

### Solución

---

### Ejercicio

----

A partir del dataset de ventas, seleccionar la categoría de Tecnología y

- Generar la serie de tiempo sobre las ventas y graficarla.
- Graficar los componentes de la serie.
- Generar un modelo de Tendencia lineal y validarlo. 

### Ejercicio

----
A partir del dataset filtrado por Tecnología, generamos la serie de tiempo y la graficamos.

In [ ]:
df = pd.read_excel("../Data/Supertienda.xls")
tecno = df.loc[df['Categoria'] == 'Tecnología']

In [ ]:
tecno.head(2)

In [ ]:
tecnogb = tecno.groupby('Fecha_pedido')['Ventas'].sum().reset_index()
tecnogb = tecnogb.set_index('Fecha_pedido')

In [ ]:
df_ventas_t = tecnogb['Ventas'].resample('MS').mean().to_frame(); 
df_ventas_t.head(2)

Graficamos la serie.

In [ ]:
plt.figure(figsize=(4,3)); df_ventas_t.plot(); plt.show()

### Ejercicio

----
Graficamos los componentes de la serie de tiempo (Tendencia, Estacionalidad y la Componente aleatoria).

In [ ]:
import statsmodels.api as sm
plt.figure(figsize=(16,10))
decomposition = sm.tsa.seasonal_decompose(df_ventas_t, model='additive')
decomposition.plot()
plt.show()

### Ejercicio

----
Generamos el modelo de Tendencia Lineal: lo ajustamos sobre train, creamos las predicciones, las graficamos sobre train y test, y evaluamos el modelo con RMSE.

Previamente definimos la variable “dummy de tiempo”, la secuencia que representa al tiempo.

In [ ]:
df_ventas_t["timeIndex"] = pd.Series(np.arange(len(df_ventas_t['Ventas'])), index=df_ventas_t.index)
df_ventas_t.timeIndex.head(3)

Hacemos los datasets de train y test. Recordar que tenemos que usar `shuffle=False`.

In [ ]:
from sklearn.model_selection import train_test_split
df_train_t, df_test_t = train_test_split(df_ventas_t, test_size=12, random_state=42, shuffle=False)

Creamos un modelo de *regresión lineal* entre "Ventas" y "timeIndex". 

In [ ]:
import statsmodels.formula.api as smf
model_linear = smf.ols(formula='Ventas ~ timeIndex', data = df_train_t).fit()

Generamos las predicciones del modelo sobre *train y test*.

In [ ]:
df_train_t["LinearTrend"] = model_linear.predict(df_train_t.timeIndex)
df_test_t["LinearTrend"] = model_linear.predict(df_test_t.timeIndex)

Chequeamos visualmente las predicciones. 

In [ ]:
fig, axes = plt.subplots(1, 2,figsize=(12,6))
df_train_t.plot(kind = "line", y = ["Ventas","LinearTrend"],ax=axes[0],title='Train');
df_test_t.plot(kind = "line", y = ["Ventas","LinearTrend"],ax=axes[1],title='Test');
plt.show()

Calculamos la performance con RMSE.

In [ ]:
def RMSE(predicted, actual):
    mse = (predicted - actual) ** 2
    rmse = np.sqrt(mse.sum() / mse.count())
    return rmse

In [ ]:
model_RMSE = RMSE(df_test_t.LinearTrend, df_test_t.Ventas)
print('Evaluacion RMSE: ',model_RMSE)

<div class="div-dhds-fondo-1"> Referencias y Material Adicional
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M3/CLASE_21_Regresion_Lineal_Simple/Presentacion/img/M3_CLASE_21_separador.png" align="center" />

</div>

### Referencias y Material Adicional

---

<a href="https://www.statsmodels.org/stable/index.html" target="_blank">statmodels</a>

<a href="https://pandas.pydata.org/pandas-docs/version/0.23/timeseries.html" target="_blank">Pandas - Time Series / Date functionality</a>

<a href="https://konstantin-rink.medium.com/your-comprehensive-guide-to-the-basics-of-time-series-modeling-f673398b5df3" target="_blank">Your comprehensive guide to the basics of Time Series Modeling</a>

<a href="https://otexts.com/fpp2/" target="_blank">Forecasting: Principles and Practice</a>

<a href="https://towardsdatascience.com/pandas-resample-tricks-you-should-know-for-manipulating-time-series-data-7e9643a7e7f3" target="_blank">Pandas resample() tricks you should know for manipulating time-series data</a>

<a href="https://towardsdatascience.com/time-series-in-python-exponential-smoothing-and-arima-processes-2c67f2a52788" target="_blank">Time Series in Python — Exponential Smoothing and ARIMA processes</a>

<a href="https://www.analyticsvidhya.com/blog/2018/02/time-series-forecasting-methods/" target="_blank">Time Series Forecasting Methods</a>

<a href="https://towardsdatascience.com/moving-averages-in-python-16170e20f6c" target="_blank">Moving averages with Python</a>